In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)   #훈련세트 80, 테스트세트 20 으로 나눔
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)  #훈련세트 80 - > 훈련 60, 검증세트 20 으로 나눔

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_target)

from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
# print(scores) -> cross_validate 는 기본적으로 5개의 숫자(5-폴드 교차 검증이 defalt) 가 담긴 키를 fit_time, scroe_time, test_score 에 딕셔너리 로 반환

import numpy as np
#print(np.mean(scores['test_score'])) -> test_score 키에 들어있는 값들 평균이 교차검증의 점수

from sklearn.model_selection import StratifiedKFold                             #cross_validate 은 사전에 훈련세트를 무작위로 섞어주지않으므로 사용한다면 분할기도 별도 지정필수
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)        #n_splits -> n 차 교차검증 지정
scores = cross_validate(dt, train_input, train_target, cv= splitter)

from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)


#랜덤 서치 이용
from scipy.stats import uniform, randint

params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1, 25),
          }

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter  = 100, n_jobs = -1, random_state = 42)          #n_iter -> 랜덤 샘플링을 매개변수 마다 100번씩 진행
gs.fit(train_input, train_target)


0.9971133028626413
0.8576923076923076
